In [1]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'pascalvoc'

In [2]:
s3_train_data = 's3://{}/{}/input/train'.format(bucket, prefix)
s3_validation_data = 's3://{}/{}/input/validation'.format(bucket, prefix)

In [3]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [5]:
from sagemaker import image_uris
region = session.boto_region_name
container = image_uris.retrieve('object-detection', region)

In [6]:
od = sagemaker.estimator.Estimator(
    container,
    sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    output_path=s3_output_location)

In [7]:
od.set_hyperparameters(base_network='resnet-50',
                       use_pretrained_model=1,
                       num_classes=20,
                       num_training_samples=16551,
                       epochs=30)

In [8]:
from sagemaker.session import TrainingInput
train_data = TrainingInput (
    s3_train_data,
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='S3Prefix')

In [9]:
validation_data = TrainingInput (
    s3_validation_data,
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='S3Prefix')

In [10]:
data_channels = {'train': train_data, 'validation': validation_data}

In [11]:
od.fit(inputs=data_channels)

2021-04-25 09:22:51 Starting - Starting the training job...
2021-04-25 09:23:14 Starting - Launching requested ML instancesProfilerReport-1619342571: InProgress
......
2021-04-25 09:24:16 Starting - Preparing the instances for training......
2021-04-25 09:25:21 Downloading - Downloading input data......
2021-04-25 09:26:15 Training - Downloading the training image...
2021-04-25 09:26:35 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[04/25/2021 09:26:38 INFO 140200834508608] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/default-input.json: {'base_network': 'vgg-16', 'use_pretrained_model': '0', 'num_classes': '', 'mini_batch_size': '32', 'epochs': '30', 'learning_rate': '0.001', 'lr_scheduler_step': '', 'lr_scheduler_factor': '0.1', 'optimizer': 'sgd', 'momentum': '0.9', 'weight_decay': '0.0005', 'overlap_threshold': '0.5', 'nms_threshold': '0.45', 'num_training_samples': '', 'im

In [12]:
od_predictor = od.deploy(initial_instance_count = 1, instance_type = 'ml.c5.2xlarge')

-------------!

In [37]:
import json
!wget -O test.jpg https://upload.wikimedia.org/wikipedia/commons/6/67/Chin_Village.jpg
file_name= 'test.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
    ne = open('n.txt','wb')
    ne.write(b)

od_predictor.content_type = 'application/x-recordio' # Use these content types to avoid error "cannot set attribute"
od_predictor.accept = 'image/jpeg'
results = od_predictor.predict(b)
response = json.loads(results)
print(response)

--2021-04-25 11:49:51--  https://upload.wikimedia.org/wikipedia/commons/6/67/Chin_Village.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 821188 (802K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>] 801.94K  --.-KB/s    in 0.07s   

2021-04-25 11:49:51 (12.0 MB/s) - ‘test.jpg’ saved [821188/821188]



AttributeError: can't set attribute

In [38]:
od_predictor.delete_endpoint()